In [3]:
from pyspark.shell import sqlContext
from pyspark.sql import SparkSession

In [4]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T

spark = (
    SparkSession.builder.appName("Queries_App")
    .config("spark.executor.memory", "1G")
    .config("spark.executor.cores", "2")
    .config("master", "spark://master:7077")
    .getOrCreate()
)

In [5]:
vehicles = spark.read.csv("/proiect/vehicle_claims.csv", inferSchema=True, header=True)

In [6]:
vehicles.createOrReplaceTempView('vehicles')

In [7]:
import pyspark

In [8]:
# 1. Selectează toate rândurile unde Gearbox = "Automatic"
spark.sql('SELECT * FROM vehicles WHERE Gearbox = "Automatic"').show()

+-------+---------+------------+--------+---------+------+--------+--------+------------+----------+---------+---------+-------+--------+--------+--------------------+--------+-------+--------------+-----------------+------------------+-----------------+-----------+
|  Maker| Genmodel| Genmodel_ID|Adv_year|Adv_month| Color|Reg_year|Bodytype|Runned_Miles|Engin_size|  Gearbox|Fuel_type|  Price|Seat_num|Door_num|               issue|issue_id|Adv_day|breakdown_date|repair_complexity|       repair_cost|     repair_hours|repair_date|
+-------+---------+------------+--------+---------+------+--------+--------+------------+----------+---------+---------+-------+--------+--------+--------------------+--------+-------+--------------+-----------------+------------------+-----------------+-----------+
|Bentley|   Arnage|        10_1|    2018|        4|Silver|  2000.0|  Saloon|       60000|      6.8L|Automatic|   Petrol|21500.0|     5.0|     4.0|    Electrical Issue|       4|     12|    2018-04-12|

In [9]:
# 2. Numără câte mașini sunt pentru fiecare tip de Gearbox
spark.sql('SELECT Gearbox, COUNT(*) AS count FROM vehicles GROUP BY Gearbox').show()

[Stage 3:=============================>                             (4 + 4) / 8]

+--------------+------+
|       Gearbox| count|
+--------------+------+
|     Automatic| 97119|
|        Hybrid|   167|
|        Manual|170832|
|Semi-Automatic|   137|
+--------------+------+



In [10]:
# 3. Media prețului (Price) pentru fiecare tip de Gearbox
spark.sql('SELECT Gearbox, AVG(Price) AS avg_price FROM vehicles GROUP BY Gearbox').show()

[Stage 6:===================================================>       (7 + 1) / 8]

+--------------+------------------+
|       Gearbox|         avg_price|
+--------------+------------------+
|     Automatic|21492.755042782566|
|        Hybrid| 92543.62275449102|
|        Manual| 8574.819694202492|
|Semi-Automatic|109941.43795620438|
+--------------+------------------+



In [11]:
# 4. Selectează toate mașinile care au repair_complexity = 3 și Gearbox este "Automatic"
spark.sql('SELECT distinct(Maker) FROM vehicles WHERE repair_complexity = 3 AND Gearbox = "Automatic"').show()


[Stage 9:===================================================>       (7 + 1) / 8]

+------------+
|       Maker|
+------------+
|    Cadillac|
|    Chrysler|
|     Ferrari|
|    Daihatsu|
|     Bentley|
|  Land Rover|
|  Alfa Romeo|
|     McLaren|
|     Lincoln|
|       Lotus|
|       Rover|
| Rolls-Royce|
|      Jaguar|
|     Porsche|
|Aston Martin|
|      Subaru|
|       Volvo|
+------------+



In [12]:
# 5.Numărul total de reparații și suma orelor de reparație pe fiecare Maker și tip de issue
query = """
SELECT 
  Maker, 
  issue, 
  COUNT(*) AS total_repairs, 
  SUM(repair_hours) AS total_repair_hours
FROM vehicles
GROUP BY Maker, issue
ORDER BY total_repairs DESC
"""
spark.sql(query).show()


+-----+--------------------+-------------+------------------+
|Maker|               issue|total_repairs|total_repair_hours|
+-----+--------------------+-------------+------------------+
| Ford|       Warning Light|         1901|100890.76638849115|
| Ford|  Transmission Issue|         1862| 55366.73960970528|
| Ford|    Electrical Issue|         1850|24105.280654733648|
| Ford|        Engine Issue|         1828|329831.52258088323|
| Ford|      Tyre Alignment|         1828| 4173.443043171217|
| Ford|  Alternator Failing|         1817| 79422.81998115567|
| Ford| Starter Motor Issue|         1812|46039.652652356555|
| Ford|     Brake Pads Worn|         1810| 15982.75249928554|
| Ford|  Sensor Malfunction|         1795| 158318.6829257151|
| Ford|Steering Wheel Sh...|         1771|10303.298949893124|
| Ford|    Radiator Leaking|         1761| 78009.85183164844|
| Ford|      Gear Box Issue|         1760|46004.813953169956|
| Ford|    Windscreen Crack|         1742| 9104.707215245651|
| Ford| 